<div style="font-size: 10px;">

# Final Evaluation: Test Set Performance

================================================================================
PURPOSE: Evaluate final models on the held-out test set
================================================================================

**CRITICAL**: This notebook evaluates on the **TEST set** which was separated
at the beginning (in 01_data_split.ipynb) and has NEVER been used for training,
model selection, feature selection, or any development decisions.

**Evaluation Protocol:**
1. Load best model/classifier combinations (based on Dev set results from
   03_train_evaluate.ipynb and 04_early_fusion.ipynb)
2. Extract features for TEST set (if not already extracted)
3. Retrain models on combined Train+Dev data (final training)
4. Evaluate on TEST set (unbiased final performance estimate)
5. Generate comprehensive final reports, plots, and summary tables

**Important Notes:**
- Test set is ONLY accessed in this notebook
- Models are retrained on Train+Dev before test evaluation
- This provides an unbiased estimate of generalization performance
- Results from this notebook represent the final competition submission metrics

================================================================================
INPUTS (What this notebook loads)
================================================================================

**From GitHub:**
- Repository code (cloned automatically if not present)
- Source modules from `src/` directory:
  - `src.storage.manager` (StorageManager)
  - `src.features.extraction` (feature extraction functions)
  - `src.models.trainer` (training and evaluation functions)
  - `src.models.classifiers` (classifier definitions)
  - `src.evaluation.metrics` (metric computation functions)
  - `src.evaluation.tables` (results table functions)
  - `src.evaluation.visualizer` (plotting functions)

**From Google Drive:**
- Dataset splits: `splits/dataset_splits.pkl`
  - Test split (ONLY accessed in this notebook, 308 samples)
  - Train and Dev splits (for combined training)
- Feature matrices (if already extracted):
  - `features/raw/X_test_{model}_{task}.npy` (may not exist yet)
  - `features/raw/X_train_{model}_{task}.npy` (from 02_feature_extraction_separate.ipynb)
  - `features/raw/X_dev_{model}_{task}.npy` (from 02_feature_extraction_separate.ipynb)

**From HuggingFace Hub:**
- Transformer models (if test features need extraction):
  - BERT, RoBERTa, DeBERTa, XLNet tokenizers and models

================================================================================
OUTPUTS (What this notebook saves)
================================================================================

**To Google Drive:**
- Test features (if extracted): `features/raw/X_test_{model}_{task}.npy`
  - For each model (bert, roberta, deberta, xlnet)
  - For each task (clarity, evasion)
  - Shape: (308_samples, 19_features)
- Final predictions: `predictions/pred_test_{model}_{classifier}_{task}.npy`
  - Hard label predictions for Test set
  - For each model/classifier/task combination
- Final probabilities: `features/probabilities/probs_test_{model}_{classifier}_{task}.npy`
  - Probability distributions for Test set
  - For each model/classifier/task combination
- Final evaluation plots: `plots/final_evaluation/{model}_{task}_{classifier}/`
  - Confusion matrices
  - Precision-Recall curves
  - ROC curves

**To GitHub:**
- Test feature metadata: `metadata/features_test_{model}_{task}.json`
  - Feature names and dimensions
  - Timestamp information
- Final results metadata: `results/FINAL_TEST_{model}_{task}.json`
  - Final test set metrics
  - Model/classifier/task information
  - Test sample counts

**Evaluation Metrics Computed and Printed:**
- Accuracy
- Macro Precision, Recall, F1
- Weighted Precision, Recall, F1
- Per-class metrics (precision, recall, F1, support)
- Cohen's Kappa
- Matthews Correlation Coefficient
- Hamming Loss
- Jaccard Score (IoU)
- Confusion Matrix

**What represents final competition submission:**
- All test set predictions and probabilities
- Final evaluation metrics (computed on 308 test samples)
- Complete evaluation results for all model/classifier/task combinations

</div>


In [ ]:
# ============================================================================
# SETUP: Repository Clone, Drive Mount, and Path Configuration
# ============================================================================
# This cell performs minimal setup required for the notebook to run:
# 1. Clones repository from GitHub (if not already present)
# 2. Mounts Google Drive for persistent data storage
# 3. Configures Python paths and initializes StorageManager
# 4. Loads test split (ONLY accessed in this notebook)

import shutil
import os
import subprocess
import time
import requests
import zipfile
import sys
from pathlib import Path
from google.colab import drive
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

# Repository configuration
repo_dir = '/content/semeval-context-tree-modular'
repo_url = 'https://github.com/EonTechie/semeval-context-tree-modular.git'
zip_url = 'https://github.com/EonTechie/semeval-context-tree-modular/archive/refs/heads/main.zip'

# Clone repository (if not already present)
if not os.path.exists(repo_dir):
    print("Cloning repository from GitHub...")
    max_retries = 2
    clone_success = False
    
    for attempt in range(max_retries):
        try:
            result = subprocess.run(
                ['git', 'clone', repo_url],
                cwd='/content',
                capture_output=True,
                text=True,
                timeout=60
            )
            if result.returncode == 0:
                print("Repository cloned successfully via git")
                clone_success = True
                break
            else:
                if attempt < max_retries - 1:
                    time.sleep(3)
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(3)
    
    # Fallback: Download as ZIP if git clone fails
    if not clone_success:
        print("Git clone failed. Downloading repository as ZIP archive...")
        zip_path = '/tmp/repo.zip'
        try:
            response = requests.get(zip_url, stream=True, timeout=60)
            response.raise_for_status()
            with open(zip_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content')
            extracted_dir = '/content/semeval-context-tree-modular-main'
            if os.path.exists(extracted_dir):
                os.rename(extracted_dir, repo_dir)
            os.remove(zip_path)
            print("Repository downloaded and extracted successfully")
        except Exception as e:
            raise RuntimeError(f"Failed to obtain repository: {e}")

# Mount Google Drive (if not already mounted)
try:
    drive.mount('/content/drive', force_remount=False)
except Exception:
    pass  # Already mounted

# Configure paths
BASE_PATH = Path('/content/semeval-context-tree-modular')
DATA_PATH = Path('/content/drive/MyDrive/semeval_data')

# Verify repository structure exists
if not BASE_PATH.exists():
    raise RuntimeError(f"Repository directory not found: {BASE_PATH}")
if not (BASE_PATH / 'src').exists():
    raise RuntimeError(f"src directory not found in repository: {BASE_PATH / 'src'}")
if not (BASE_PATH / 'src' / 'storage' / 'manager.py').exists():
    raise RuntimeError(f"Required file not found: {BASE_PATH / 'src' / 'storage' / 'manager.py'}")

# Add repository to Python path
sys.path.insert(0, str(BASE_PATH))

# Verify imports work
try:
    from src.storage.manager import StorageManager
    from src.features.extraction import featurize_hf_dataset_in_batches_v2
    from src.models.classifiers import get_classifier_dict
    from src.evaluation.metrics import compute_all_metrics, print_classification_report
    from src.evaluation.tables import print_results_table
    from src.evaluation.visualizer import visualize_all_evaluation
except ImportError as e:
    raise ImportError(
        f"Failed to import required modules. "
        f"Repository path: {BASE_PATH}, "
        f"Python path: {sys.path[:3]}, "
        f"Error: {e}"
    )

# Initialize StorageManager
storage = StorageManager(
    base_path=str(BASE_PATH),
    data_path=str(DATA_PATH),
    github_path=str(BASE_PATH)
)

# Test splits will be loaded per-task in the evaluation loop
# Clarity and Evasion have different test splits (Evasion uses majority voting)

print("Setup complete")
print(f"  Repository: {BASE_PATH}")
print(f"  Data storage: {DATA_PATH}")
print(f"\nCRITICAL: Test sets will be loaded per-task (task-specific splits)")
print("         Clarity and Evasion have different test splits due to majority voting")
print("         These sets have NEVER been used for training or development!")


In [ ]:
# ============================================================================
# CONFIGURE MODELS, TASKS, AND CLASSIFIERS FOR FINAL EVALUATION
# ============================================================================
# Defines the models, tasks, and classifiers for final evaluation
# NOTE: In practice, you should select best model/classifier based on Dev set
# results. For comprehensive comparison, all combinations are evaluated here.

from src.models.final_evaluation import run_final_evaluation

MODELS = ['bert', 'bert_political', 'bert_ambiguity', 'roberta', 'deberta', 'xlnet']
TASKS = ['clarity', 'evasion']

# Label mappings for each task
CLARITY_LABELS = ['Ambivalent', 'Clear Non-Reply', 'Clear Reply']
EVASION_LABELS = ['Claims ignorance', 'Clarification', 'Declining to answer', 
                  'Deflection', 'Dodging', 'Explicit', 
                  'General', 'Implicit', 'Partial/half-answer']

# Model configurations (HuggingFace model names)
MODEL_CONFIGS = {
    'bert': 'bert-base-uncased',
    'bert_political': 'bert-base-uncased',  # TODO: Replace with actual political discourse BERT model
    'bert_ambiguity': 'bert-base-uncased',  # TODO: Replace with actual ambiguity-focused BERT model
    'roberta': 'roberta-base',
    'deberta': 'microsoft/deberta-v3-base',
    'xlnet': 'xlnet-base-cased'
}

# Max sequence lengths for each model
MODEL_MAX_LENGTHS = {
    'bert': 512,
    'bert_political': 512,
    'bert_ambiguity': 512,
    'roberta': 512,
    'deberta': 512,
    'xlnet': 1024
}

# Label lists for each task
LABEL_LISTS = {
    'clarity': CLARITY_LABELS,
    'evasion': EVASION_LABELS
}

# Initialize classifiers with fixed random seed for reproducibility
classifiers = get_classifier_dict(random_state=42)

# Configure device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Configuration for final evaluation:")
print(f"  Models: {MODELS}")
print(f"  Tasks: {TASKS}")
print(f"  Classifiers: {list(classifiers.keys())}")
print(f"  Device: {device}")
print(f"\nNOTE: Evaluating all model/classifier combinations on TEST set")
print("      In practice, select best combination based on Dev set results")


In [ ]:
# ============================================================================
# FINAL EVALUATION: YARIŞMAYA UYGUN TEK FONKSİYON ÇAĞRISI
# ============================================================================
# Bu fonksiyon tüm final evaluation işlemlerini yapar:
# 1. Test feature'larını extract eder (yoksa) veya Drive'dan yükler
# 2. Train+Dev üzerinde tüm model×classifier kombinasyonlarını eğitir
# 3. Test setinde tahmin yapar ve metrikleri hesaplar
# 4. Hierarchical evaluation yapar (evasion → clarity)
# 5. Sonuçları kaydeder ve görselleştirir

results = run_final_evaluation(
    storage=storage,
    models=MODELS,
    tasks=TASKS,
    model_configs=MODEL_CONFIGS,
    model_max_lengths=MODEL_MAX_LENGTHS,
    label_lists=LABEL_LISTS,
    device=device,
    classifiers=classifiers,
    random_state=42,
    batch_size=8,
    save_results=True,    # Sonuçları kaydet
    create_plots=True     # Plot'ları oluştur
)

print("\n" + "="*80)
print("FINAL EVALUATION COMPLETE")
print("="*80)
print("\nResults are available in the 'results' dictionary:")
print("  - results['final_results']: Model/task/classifier predictions and metrics")
print("  - results['hierarchical_results']: Hierarchical evasion→clarity results")
print("\nAll predictions and plots have been saved to Google Drive.")
